# ML Model Deployment Project

This notebook will help you build, save, and deploy a machine learning model using Streamlit.

In [1]:
# Install required packages
import subprocess
import sys

# Install packages one by one
packages = ['pandas', 'scikit-learn', 'joblib', 'streamlit', 'matplotlib', 'seaborn']

subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)
print("All packages installed successfully")

All packages installed successfully


In [2]:
# Import all required libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

print("All libraries imported successfully")

All libraries imported successfully


In [3]:
# Load the Iris dataset from sklearn
iris = load_iris()

# Create a DataFrame from the dataset
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['target'] = iris.target

# Display basic information about the dataset
print("Dataset shape:", data.shape)
print("Dataset columns:", data.columns.tolist())
print("First 5 rows:")
print(data.head())

Dataset shape: (150, 5)
Dataset columns: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target']
First 5 rows:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [4]:
# Prepare the data for machine learning
# Separate features and target
X = data.drop('target', axis=1)
y = data['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)
print("Target classes:", iris.target_names)

Training set size: (120, 4)
Testing set size: (30, 4)
Target classes: ['setosa' 'versicolor' 'virginica']


In [5]:
# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Model trained successfully")
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

Model trained successfully
Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [6]:
# Save the trained model
model_filename = 'iris_model.pkl'
joblib.dump(model, model_filename)

# Also save the feature names and target names for later use
model_info = {
    'feature_names': iris.feature_names,
    'target_names': iris.target_names
}
joblib.dump(model_info, 'model_info.pkl')

print("Model saved as:", model_filename)
print("Model info saved as: model_info.pkl")

Model saved as: iris_model.pkl
Model info saved as: model_info.pkl


In [7]:
# Load the saved model
loaded_model = joblib.load('iris_model.pkl')
loaded_model_info = joblib.load('model_info.pkl')

# Test the loaded model with sample data
sample_data = [[5.1, 3.5, 1.4, 0.2]]  # Sample iris flower measurements
prediction = loaded_model.predict(sample_data)
prediction_proba = loaded_model.predict_proba(sample_data)

print("Model loaded successfully")
print("Sample input:", sample_data[0])
print("Predicted class:", loaded_model_info['target_names'][prediction[0]])
print("Prediction probabilities:", prediction_proba[0])

Model loaded successfully
Sample input: [5.1, 3.5, 1.4, 0.2]
Predicted class: setosa
Prediction probabilities: [1. 0. 0.]


C:\Users\uday2\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\uday2\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [8]:
# Create Streamlit app code
streamlit_app_code = '''
import streamlit as st
import joblib
import numpy as np

# Load the model and model info
@st.cache_resource
def load_model():
    model = joblib.load('iris_model.pkl')
    model_info = joblib.load('model_info.pkl')
    return model, model_info

# Load model
model, model_info = load_model()

# App title
st.title('Iris Flower Classifier')

# App description
st.write('This app predicts the species of iris flowers based on their measurements.')

# Create input widgets
st.sidebar.header('Input Features')

sepal_length = st.sidebar.slider('Sepal Length (cm)', 4.0, 8.0, 5.0)
sepal_width = st.sidebar.slider('Sepal Width (cm)', 2.0, 4.5, 3.0)
petal_length = st.sidebar.slider('Petal Length (cm)', 1.0, 7.0, 4.0)
petal_width = st.sidebar.slider('Petal Width (cm)', 0.1, 2.5, 1.0)

# Create input array
input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])

# Display input values
st.subheader('Input Values')
st.write('Sepal Length:', sepal_length)
st.write('Sepal Width:', sepal_width)
st.write('Petal Length:', petal_length)
st.write('Petal Width:', petal_width)

# Make prediction
prediction = model.predict(input_data)
prediction_proba = model.predict_proba(input_data)

# Display prediction
st.subheader('Prediction')
predicted_species = model_info['target_names'][prediction[0]]
st.write('Predicted Species:', predicted_species)

# Display prediction probabilities
st.subheader('Prediction Probabilities')
for i, species in enumerate(model_info['target_names']):
    st.write(f'{species}: {prediction_proba[0][i]:.3f}')
'''

# Save the Streamlit app to a Python file
with open('streamlit_app.py', 'w') as f:
    f.write(streamlit_app_code)

print("Streamlit app created as: streamlit_app.py")

Streamlit app created as: streamlit_app.py


In [9]:
# Create requirements.txt file
requirements = '''pandas==2.0.3
scikit-learn==1.3.0
joblib==1.3.1
streamlit==1.25.0
numpy==1.24.3
matplotlib==3.7.2
seaborn==0.12.2
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("Requirements file created: requirements.txt")

Requirements file created: requirements.txt


In [11]:
# Create README.md file
readme_content = '''# Iris Flower Classification Project

## Overview
This project builds a machine learning model to classify iris flowers into three species: setosa, versicolor, and virginica. The model uses flower measurements to make predictions.

## Features
- Machine learning model using Random Forest classifier
- Streamlit web application for easy predictions
- Model accuracy: 100% on test data

## Setup Instructions

1. Install Python 3.8 or higher
2. Install required packages:
   ```
   pip install -r requirements.txt
   ```
3. Run the Streamlit app:
   ```
   streamlit run streamlit_app.py
   ```

## Input Examples
The model expects four measurements:
- Sepal Length: 4.0 - 8.0 cm
- Sepal Width: 2.0 - 4.5 cm  
- Petal Length: 1.0 - 7.0 cm
- Petal Width: 0.1 - 2.5 cm

## Sample Predictions
- Input: [5.1, 3.5, 1.4, 0.2] -> Prediction: Setosa
- Input: [6.0, 3.0, 4.5, 1.5] -> Prediction: Versicolor
- Input: [6.5, 3.0, 5.5, 2.0] -> Prediction: Virginica

## Files
- iris_model.pkl: Trained machine learning model
- model_info.pkl: Model metadata and class names
- streamlit_app.py: Web application
- requirements.txt: Required Python packages
- ml_model_deployment.ipynb: Complete development notebook

## Deployment
This app can be deployed on cloud platforms like:
- Streamlit Cloud
- Heroku
- AWS
- Google Cloud Platform

For Streamlit Cloud deployment:
1. Upload files to GitHub repository
2. Connect repository to Streamlit Cloud
3. Deploy with one click
'''

with open('README.md', 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("README file created: README.md")

README file created: README.md


In [12]:
# Project Summary
print("=== PROJECT COMPLETED SUCCESSFULLY ===")
print("Files created:")
print("1. iris_model.pkl - Trained machine learning model")
print("2. model_info.pkl - Model metadata")
print("3. streamlit_app.py - Web application")
print("4. requirements.txt - Dependencies list")
print("5. README.md - Project documentation")
print()
print("To run the Streamlit app:")
print("1. Open terminal in this directory")
print("2. Run: streamlit run streamlit_app.py")
print("3. Open browser to the provided URL")
print()
print("Model Performance:")
print("- Accuracy: 100% on test data")
print("- Dataset: Iris flowers (150 samples)")
print("- Features: 4 measurements")
print("- Classes: 3 species")

=== PROJECT COMPLETED SUCCESSFULLY ===
Files created:
1. iris_model.pkl - Trained machine learning model
2. model_info.pkl - Model metadata
3. streamlit_app.py - Web application
4. requirements.txt - Dependencies list
5. README.md - Project documentation

To run the Streamlit app:
1. Open terminal in this directory
2. Run: streamlit run streamlit_app.py
3. Open browser to the provided URL

Model Performance:
- Accuracy: 100% on test data
- Dataset: Iris flowers (150 samples)
- Features: 4 measurements
- Classes: 3 species


In [13]:
# Verify all files are created and show their sizes
import os

files_to_check = [
    'iris_model.pkl',
    'model_info.pkl', 
    'streamlit_app.py',
    'requirements.txt',
    'README.md',
    'test_model.py',
    'PROJECT_REPORT.md'
]

print("Final verification - All project files:")
print("=" * 50)

for file in files_to_check:
    size = os.path.getsize(file) if os.path.exists(file) else 0
    status = "✓" if os.path.exists(file) else "✗"
    print(f"{status} {file:<25} ({size} bytes)")

print("\nProject is ready for deployment!")
print("Run this command to start the web app:")
print("streamlit run streamlit_app.py")

Final verification - All project files:
✓ iris_model.pkl            (187297 bytes)
✓ model_info.pkl            (458 bytes)
✓ streamlit_app.py          (1603 bytes)
✓ requirements.txt          (121 bytes)
✓ README.md                 (1510 bytes)
✓ test_model.py             (883 bytes)
✓ PROJECT_REPORT.md         (4231 bytes)

Project is ready for deployment!
Run this command to start the web app:
streamlit run streamlit_app.py
